In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [54]:
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
from hanspell import spell_checker
from konlpy.tag import Mecab

In [9]:
# GPU
device = torch.device("cuda:0")

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [11]:
# google drive mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [17]:
# 데이터 불러오기

import pandas as pd

dataset = pd.read_csv('/gdrive/My Drive/KoBERT/data/data_final.csv')
# dataset = pd.read_csv('/gdrive/My Drive/KoBERT/data/data_mecab.csv')
# dataset = pd.read_csv('/gdrive/My Drive/KoBERT/data/data_1.csv')

In [18]:
dataset

,sentence,emotion
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,화남
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,비참함
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,비참함
3,꼭 롤러코스터 타는 것 같아요.,긴장됨
4,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,우울함
...,...,...
51235,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,기쁨
51236,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,지루함
51237,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,비참함
51238,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,화남


In [19]:
# 전처리 완료되면 삭제

drop_upset_index = dataset[dataset.emotion == '화남'].sample(n=2000, random_state=0).index
drop_distressed_index = dataset[dataset.emotion == '괴로움'].sample(n=500, random_state=0).index

dataset = dataset.drop(drop_upset_index, axis=0)
dataset = dataset.drop(drop_distressed_index, axis=0)

dataset.loc[dataset['emotion'] == '지루함', 'emotion'] = '피로함'

In [20]:
dataset.emotion.value_counts()

화남     7612
괴로움    7596
기쁨     7572
우울함    7234
긴장됨    5526
비참함    5355
놀람     3163
피로함    2567
중립     2115
Name: emotion, dtype: int64

In [31]:
# 감정 라벨링

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(dataset['emotion'])
dataset['emotion'] = encoder.transform(dataset['emotion'])
dataset

,sentence,emotion
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,8
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,4
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,4
3,꼭 롤러코스터 타는 것 같아요.,2
4,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,5
...,...,...
51234,학교 친구들이 나만 계속 따돌려서 너무 무서워.,8
51235,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,1
51236,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,7
51237,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,4


In [32]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '괴로움',
 1: '기쁨',
 2: '긴장됨',
 3: '놀람',
 4: '비참함',
 5: '우울함',
 6: '중립',
 7: '피로함',
 8: '화남'}

In [33]:
data_list = []
for q, label in zip(dataset['sentence'], dataset['emotion']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [34]:
# train/test 분리

from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size=0.2, random_state=42)
print("train:", len(train))
print("test:", len(test))

train: 38992
test: 9748


In [35]:
train[:5]

[['하고 싶은 것도 딱히 없고, 빨리 취직하고 싶어서 상고에 갔어요.', '6'],
 ['나 어제 여자친구랑 헤어졌어. 너무 화가 나', '8'],
 ['가해자들이 나에 대한 괴롭힘을 멈춰주겠다며 다른 친구를 따돌리라고 해.', '3'],
 ['너무 슬프고 우울해.', '5'],
 ['갈수록 체력이 떨어져서 불안한 마음이 들어.', '2']]

In [36]:
test[:5]

[['지난번 사고난 곳이 비만 오면 쑤시네.', '0'],
 ['근데 안 갔어. 아무래도 정신과는 좀 그렇잖아.', '6'],
 ['남편이 어젯밤 또 술 취해서 들어왔어. 술 끊겠다는 약속을 믿은 내가 바보였어.', '8'],
 ['아이를 위해서 직장에 다니는데 아이 케어를 더 못하는 것 같아.', '3'],
 ['주변 친한 지인이 죽고 나니 많이 혼란스럽네.', '0']]

In [55]:
tokenizer = get_tokenizer()
# sp = SentencepieceTokenizer(tokenizer)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [38]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [72]:
# Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [62]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [49]:
train[:5]

[['하고 싶은 것도 딱히 없고, 빨리 취직하고 싶어서 상고에 갔어요.', '6'],
 ['나 어제 여자친구랑 헤어졌어. 너무 화가 나', '8'],
 ['가해자들이 나에 대한 괴롭힘을 멈춰주겠다며 다른 친구를 따돌리라고 해.', '3'],
 ['너무 슬프고 우울해.', '5'],
 ['갈수록 체력이 떨어져서 불안한 마음이 들어.', '2']]

In [40]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [59]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      
model = BERTClassifier(bertmodel, dr_rate=0.1).to(device)

In [42]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [43]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [44]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/610 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6115059852600098 train acc 0.421875
epoch 1 batch id 201 loss 1.2084373235702515 train acc 0.5220771144278606
epoch 1 batch id 401 loss 1.4798892736434937 train acc 0.5399392144638404
epoch 1 batch id 601 loss 1.4179673194885254 train acc 0.5500207986688852
epoch 1 train acc 0.5498719262295082


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/153 [00:00<?, ?it/s]

...

epoch 6 batch id 1 loss 0.3276999294757843 train acc 0.890625
epoch 6 batch id 201 loss 0.5576307773590088 train acc 0.8429726368159204
epoch 6 batch id 401 loss 0.5666828155517578 train acc 0.8514650872817955
epoch 6 batch id 601 loss 0.6463155150413513 train acc 0.8559952163061564
epoch 6 train acc 0.8561731557377049


  0%|          | 0/153 [00:00<?, ?it/s]

epoch 6 test acc 0.6075163398692811


  0%|          | 0/610 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.15223874151706696 train acc 0.96875
epoch 7 batch id 201 loss 0.3334677815437317 train acc 0.8861940298507462


In [64]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("괴로움")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 2:
                test_eval.append("긴장됨")
            elif np.argmax(logits) == 3:
                test_eval.append("놀람")
            elif np.argmax(logits) == 4:
                test_eval.append("비참함")
            elif np.argmax(logits) == 5:
                test_eval.append("우울함")
            elif np.argmax(logits) == 6:
                test_eval.append("중립")
            elif np.argmax(logits) == 7:
                test_eval.append("피로함")
            elif np.argmax(logits) == 8:
                test_eval.append("화남")

        print(">> 입력하신 내용에서 " + test_eval[0] + "이 느껴집니다.\n")

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [65]:
from konlpy.tag import Mecab

m = Mecab()
remove_tags = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'ETN', 'ETM']

def remove_mecab(sentence):
    result = ''
    removed = [x[0] for x in m.pos(sentence) if x[1] not in remove_tags]
    result = ' '.join(removed)
    return result

In [ ]:
m.pos('친구는 여행갔는데 나는 일만해')

[('친구', 'NNG'),
 ('는', 'JX'),
 ('여행', 'NNG'),
 ('갔', 'VV+EP'),
 ('는데', 'EC'),
 ('나', 'NP'),
 ('는', 'JX'),
 ('일', 'NNG'),
 ('만', 'JX'),
 ('해', 'VV+EC')]

In [66]:
s1 = '어제도 오늘같고 오늘도 내일같고 매일매일이 똑같은 상황. 옆에 있는 친구는 여행도 가고 신나게 잘 사는데, 나는 매일 힘들고 지치네'
s2 = '어제도 오늘같고 오늘도 내일같고 매일매일이 똑같은 상황이야 옆에 있는 친구는 여행도 가고 신나게 잘 사는데, 나는 매일 힘들고 지치네'

In [67]:
sentence = s1.split('.')
for i in range(len(sentence)):
  if sentence[i] != '':
    removed_sentence = ''
    removed_sentence = remove_mecab(sentence[i])
    predict(removed_sentence)

>> 입력하신 내용에서 괴로움이 느껴집니다.

>> 입력하신 내용에서 놀람이 느껴집니다.



In [68]:
removed_sentence2 = remove_mecab(s2)
print(m.pos(s2))
print(removed_sentence2)
predict(removed_sentence2)

[('어제', 'NNG'), ('도', 'JX'), ('오늘', 'MAG'), ('같', 'VA'), ('고', 'EC'), ('오늘', 'NNG'), ('도', 'JX'), ('내일', 'MAG'), ('같', 'VA'), ('고', 'EC'), ('매일', 'MAG'), ('매일', 'NNG'), ('이', 'JKS'), ('똑같', 'VA'), ('은', 'ETM'), ('상황', 'NNG'), ('이', 'VCP'), ('야', 'EC'), ('옆', 'NNG'), ('에', 'JKB'), ('있', 'VV'), ('는', 'ETM'), ('친구', 'NNG'), ('는', 'JX'), ('여행', 'NNG'), ('도', 'JX'), ('가', 'VV'), ('고', 'EC'), ('신나', 'VV'), ('게', 'EC'), ('잘', 'MAG'), ('사', 'VV'), ('는데', 'EC'), (',', 'SC'), ('나', 'NP'), ('는', 'JX'), ('매일', 'MAG'), ('힘들', 'VA'), ('고', 'EC'), ('지치', 'VV'), ('네', 'EC')]
어제 오늘 같 고 오늘 내일 같 고 매일 매일 똑같 상황 이 야 옆 있 친구 여행 가 고 신나 게 잘 사 는데 , 나 매일 힘들 고 지치 네
>> 입력하신 내용에서 괴로움이 느껴집니다.



In [69]:
while True:
    input_sentence = input("챗봇에게 하고싶은 말을 입력해주세요 : ")
    if input_sentence == "종료" :
        break
    # predict(input_sentence)
    sentence = input_sentence.split('.')
    for i in range(len(sentence)):
      if sentence[i] != '':
        removed_sentence = ''
        removed_sentence = remove_mecab(sentence[i])
        # print(removed_sentence)
        predict(removed_sentence)
        
    print("\n")

챗봇에게 하고싶은 말을 입력해주세요 : 오늘 정말 힘든 하루였어
>> 입력하신 내용에서 화남이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 망했다
>> 입력하신 내용에서 괴로움이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 아침부터 차도 막히고 일도 많고 힘들어
>> 입력하신 내용에서 놀람이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 친구는 여행갔는데 나는 일만 해
>> 입력하신 내용에서 화남이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 하루종일 회의가 연속이네
>> 입력하신 내용에서 괴로움이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 멀 잘못 먹었는지 계속 속이 더부룩해
>> 입력하신 내용에서 놀람이 느껴집니다.





KeyboardInterrupt: ignored